In [40]:
#random forest is a bragging technique where it use use decision tree as base model
# in random forest multiple desision tree run parallel for increasing accuracy
#row sampling , feature sampling is done and use to train these DT
# in classification- majority voting
#regressor: avg of output
#random forest is use for preventing overfitting of data


In [41]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
%matplotlib inline
warnings.filterwarnings("ignore")

In [42]:
df=pd.read_csv("Travel.csv")
df.head()

,CustomerID,ProdTaken,Age,TypeofContact,CityTier,DurationOfPitch,Occupation,Gender,NumberOfPersonVisiting,NumberOfFollowups,ProductPitched,PreferredPropertyStar,MaritalStatus,NumberOfTrips,Passport,PitchSatisfactionScore,OwnCar,NumberOfChildrenVisiting,Designation,MonthlyIncome
0,200000,1,41.0,Self Enquiry,3,6.0,Salaried,Female,3,3.0,Deluxe,3.0,Single,1.0,1,2,1,0.0,Manager,20993.0
1,200001,0,49.0,Company Invited,1,14.0,Salaried,Male,3,4.0,Deluxe,4.0,Divorced,2.0,0,3,1,2.0,Manager,20130.0
2,200002,1,37.0,Self Enquiry,1,8.0,Free Lancer,Male,3,4.0,Basic,3.0,Single,7.0,1,3,0,0.0,Executive,17090.0
3,200003,0,33.0,Company Invited,1,9.0,Salaried,Female,2,3.0,Basic,3.0,Divorced,2.0,1,5,1,1.0,Executive,17909.0
4,200004,0,NaN,Self Enquiry,1,8.0,Small Business,Male,2,3.0,Basic,4.0,Divorced,1.0,0,5,1,0.0,Executive,18468.0


In [43]:
# data cleaning
#checking for missing value,duplicate,datatype
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4888 entries, 0 to 4887
Data columns (total 20 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   CustomerID                4888 non-null   int64  
 1   ProdTaken                 4888 non-null   int64  
 2   Age                       4662 non-null   float64
 3   TypeofContact             4863 non-null   object 
 4   CityTier                  4888 non-null   int64  
 5   DurationOfPitch           4637 non-null   float64
 6   Occupation                4888 non-null   object 
 7   Gender                    4888 non-null   object 
 8   NumberOfPersonVisiting    4888 non-null   int64  
 9   NumberOfFollowups         4843 non-null   float64
 10  ProductPitched            4888 non-null   object 
 11  PreferredPropertyStar     4862 non-null   float64
 12  MaritalStatus             4888 non-null   object 
 13  NumberOfTrips             4748 non-null   float64
 14  Passport

In [44]:
df.isnull().sum()

CustomerID                    0
ProdTaken                     0
Age                         226
TypeofContact                25
CityTier                      0
DurationOfPitch             251
Occupation                    0
Gender                        0
NumberOfPersonVisiting        0
NumberOfFollowups            45
ProductPitched                0
PreferredPropertyStar        26
MaritalStatus                 0
NumberOfTrips               140
Passport                      0
PitchSatisfactionScore        0
OwnCar                        0
NumberOfChildrenVisiting     66
Designation                   0
MonthlyIncome               233
dtype: int64

In [45]:
df['Gender'].value_counts()
df['Gender']=df['Gender'].replace("Fe Male",'Female')

In [46]:
df['Gender'].value_counts()

Gender
Male      2916
Female    1972
Name: count, dtype: int64

In [47]:
df['MaritalStatus'].value_counts()

MaritalStatus
Married      2340
Divorced      950
Single        916
Unmarried     682
Name: count, dtype: int64

In [48]:
df['MaritalStatus']=df['MaritalStatus'].replace("Unmarried","Single")
df['MaritalStatus'].value_counts()

MaritalStatus
Married     2340
Single      1598
Divorced     950
Name: count, dtype: int64

In [49]:
#handling missing value
#features with na
features_na=[features for features in df.columns if df[features].isnull().sum()>=1]

In [50]:
df[features_na].select_dtypes(exclude='object').describe()

,Age,DurationOfPitch,NumberOfFollowups,PreferredPropertyStar,NumberOfTrips,NumberOfChildrenVisiting,MonthlyIncome
count,4662.000000,4637.000000,4843.000000,4862.000000,4748.000000,4822.000000,4655.000000
mean,37.622265,15.490835,3.708445,3.581037,3.236521,1.187267,23619.853491
std,9.316387,8.519643,1.002509,0.798009,1.849019,0.857861,5380.698361
min,18.000000,5.000000,1.000000,3.000000,1.000000,0.000000,1000.000000
25%,31.000000,9.000000,3.000000,3.000000,2.000000,1.000000,20346.000000
50%,36.000000,13.000000,4.000000,3.000000,3.000000,1.000000,22347.000000
75%,44.000000,20.000000,4.000000,4.000000,4.000000,2.000000,25571.000000
max,61.000000,127.000000,6.000000,5.000000,22.000000,3.000000,98678.000000


In [51]:
#filling nan value
#age
df.Age.fillna(df.Age.median(),inplace=True)
#type of contact
df.TypeofContact.fillna(df.TypeofContact.mode()[0],inplace=True)

#duration of pitch
df.DurationOfPitch.fillna(df.DurationOfPitch.median(),inplace=True)

#number of followup
df.NumberOfFollowups.fillna(df.NumberOfFollowups.mode()[0],inplace=True)

#preferred property star
df.PreferredPropertyStar.fillna(df.PreferredPropertyStar.mode()[0],inplace=True)

#number of trip
df.NumberOfTrips.fillna(df.NumberOfTrips.median(),inplace=True)

#child visiting
df.NumberOfChildrenVisiting.fillna(df.NumberOfChildrenVisiting.mode()[0],inplace=True)

#income
df.MonthlyIncome.fillna(df.MonthlyIncome.median(),inplace=True)

In [52]:
df.isnull().sum()

CustomerID                  0
ProdTaken                   0
Age                         0
TypeofContact               0
CityTier                    0
DurationOfPitch             0
Occupation                  0
Gender                      0
NumberOfPersonVisiting      0
NumberOfFollowups           0
ProductPitched              0
PreferredPropertyStar       0
MaritalStatus               0
NumberOfTrips               0
Passport                    0
PitchSatisfactionScore      0
OwnCar                      0
NumberOfChildrenVisiting    0
Designation                 0
MonthlyIncome               0
dtype: int64

In [53]:
df.drop('CustomerID',axis=1,inplace=True)

In [54]:
#feature eng
df['total_visiting']=df['NumberOfPersonVisiting']=df['NumberOfChildrenVisiting']
df.drop(columns=['NumberOfChildrenVisiting','NumberOfPersonVisiting'],axis=1,inplace=True)

In [55]:
#numerical features
num_feature=[feature for feature in df.columns if df[feature].dtype != 'O']
print("number ofnumerical featur",len(num_feature))

number ofnumerical featur 12


In [56]:
#catagory
cat_feature=[feature for feature in df.columns if df[feature].dtype == 'O']
print("number of catogory featur",len(cat_feature))

number of catogory featur 6


In [57]:
#dependent and independent feature
X=df.drop('ProdTaken',axis=1)
y=df['ProdTaken']

In [58]:
y.value_counts()
#imbalane

ProdTaken
0    3968
1     920
Name: count, dtype: int64

In [59]:
#train test split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)


In [60]:
#encoding and scaling
cat_feature=X.select_dtypes(include="object").columns
num_feature=X.select_dtypes(exclude="object").columns
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.compose import ColumnTransformer

num_tranform=StandardScaler()
oh_transform=OneHotEncoder(drop='first')

preprocessor=ColumnTransformer(
   [ ("OneHotEncoder",oh_transform,cat_feature),
    ("tandardScaler",num_tranform,num_feature)]
)


In [61]:
X_train=pd.DataFrame(preprocessor.fit_transform(X_train))
X_test=preprocessor.transform(X_test)

In [62]:
X_train.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25
0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,-0.157926,-0.7214,-1.020350,1.284279,-0.725271,-0.127737,-0.632399,0.679690,0.782966,-0.382245,-0.218050
1,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.508072,-0.7214,0.690023,0.282777,-0.725271,1.511598,-0.632399,0.679690,0.782966,-0.459799,0.956297
2,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.396069,-0.7214,-1.020350,0.282777,1.771041,0.418708,-0.632399,0.679690,0.782966,-0.245196,-0.218050
3,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.064073,-0.7214,-1.020350,1.284279,-0.725271,-0.127737,-0.632399,1.408395,-1.277194,0.213475,-0.218050
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.730071,-0.7214,2.400396,-1.720227,-0.725271,1.511598,-0.632399,-0.049015,-1.277194,-0.024889,2.130645


In [63]:
#model training
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report,recall_score,f1_score,roc_auc_score,precision_score

In [64]:
models={
    'randomforest':RandomForestClassifier(),
    'decision tree':DecisionTreeClassifier()
}
for i in range (len(list(models))):
    #model training
    model=list(models.values())[i]
    model.fit(X_train,y_train)
    # model pred
    y_train_pred=model.predict(X_train)
    y_test_pred=model.predict(X_test)

    #prefomance fot training
    model_train_acc=accuracy_score(y_train,y_train_pred)
    model_train_f1=f1_score(y_train,y_train_pred,average='weighted')
    model_train_pre=precision_score(y_train,y_train_pred)
    model_train_recall=recall_score(y_train,y_train_pred)
    model_train_roc=roc_auc_score(y_train,y_train_pred)

    #prefomance fot training
    model_test_acc=accuracy_score(y_test,y_test_pred)
    model_test_f1=f1_score(y_test,y_test_pred,average='weighted')
    model_test_pre=precision_score(y_test,y_test_pred)
    model_test_recall=recall_score(y_test,y_test_pred)
    model_test_roc=roc_auc_score(y_test,y_test_pred)

    #print training score
    print("model :",list(models.keys())[i])
    print("preformance on training set")
    print("accuracy :{:.4f}".format(model_train_acc))
    print("f1 Score:{:.4f}".format(model_train_f1))
    print("precision score :{:.4f}".format(model_train_pre))
    print("Recall Score :{:.4f}".format(model_train_recall))
    print("ROC Score{:.4f}".format(model_train_roc))
    print('--------------------------------------------------------------------')
    #print testing score
    print("preformance on testing set")
    print("accuracy :{:.4f}".format(model_test_acc))
    print("f1 Score:{:.4f}".format(model_test_f1))
    print("precision score :{:.4f}".format(model_test_pre))
    print("Recall Score :{:.4f}".format(model_test_recall))
    print("ROC Score{:.4f}".format(model_test_roc))

    print("===========================================================================")
    print("\n")





model : randomforest
preformance on training set
accuracy :1.0000
f1 Score:1.0000
precision score :1.0000
Recall Score :1.0000
ROC Score1.0000
--------------------------------------------------------------------
preformance on testing set
accuracy :0.9264
f1 Score:0.9204
precision score :0.9685
Recall Score :0.6440
ROC Score0.8194


model : decision tree
preformance on training set
accuracy :1.0000
f1 Score:1.0000
precision score :1.0000
Recall Score :1.0000
ROC Score1.0000
--------------------------------------------------------------------
preformance on testing set
accuracy :0.9192
f1 Score:0.9195
precision score :0.7887
Recall Score :0.8010
ROC Score0.8745




In [69]:
#hyper perameter tuneing
from sklearn.model_selection import RandomizedSearchCV
rf_params={"max_depth":[5,10,15,None,8],
           "max_features": ['sqrt', 'log2', 5, 7, 8],
           "min_samples_split":[2,8,15,20],
           "n_estimators":[100,200,500,1000]}

In [71]:
randomcvmodels=[("RF",RandomForestClassifier(),rf_params)]

model_param={}
for name,model,param in randomcvmodels:
    random=RandomizedSearchCV(
        estimator=model,
        param_distributions=param,
        n_iter=100,
        verbose=2,
        cv=3
    )
    random.fit(X_train,y_train)
    model_param[name]=random.best_params_

for modelname in model_param:
    print(f"best parameter for {modelname}  ::")
    print(model_param[modelname])



Fitting 5 folds for each of 100 candidates, totalling 500 fits
[CV] END max_depth=None, max_features=log2, min_samples_split=2, n_estimators=100; total time=   0.2s
[CV] END max_depth=None, max_features=log2, min_samples_split=2, n_estimators=100; total time=   0.2s
[CV] END max_depth=None, max_features=log2, min_samples_split=2, n_estimators=100; total time=   0.2s
[CV] END max_depth=None, max_features=log2, min_samples_split=2, n_estimators=100; total time=   0.2s
[CV] END max_depth=None, max_features=log2, min_samples_split=2, n_estimators=100; total time=   0.2s
[CV] END max_depth=5, max_features=8, min_samples_split=2, n_estimators=500; total time=   0.9s
[CV] END max_depth=5, max_features=8, min_samples_split=2, n_estimators=500; total time=   0.9s
[CV] END max_depth=5, max_features=8, min_samples_split=2, n_estimators=500; total time=   0.9s
[CV] END max_depth=5, max_features=8, min_samples_split=2, n_estimators=500; total time=   0.9s
[CV] END max_depth=5, max_features=8, min_s